In [ ]:
pip install SpeechRecognition pydub

In [ ]:
pip install openai-whisper


In [ ]:
import whisper

model = whisper.load_model("small")  # or "base", "medium"
result = model.transcribe("Recording.m4a")
print(result["text"])

In [ ]:
import os
import re
import torch
import whisper
from transformers import pipeline
from transformers import AutoTokenizer

In [ ]:
# Force GPU if available
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load summariser and sentiment analyser
summariser = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=0 if DEVICE == "cuda" else -1
)
sentiment_analyser = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=0 if DEVICE == "cuda" else -1
)

# Load tokenizer for precise truncation
sentiment_tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [ ]:
def safe_filename(text: str, max_len: int = 50) -> str:
    raw = "_".join(text.strip().split())
    safe = re.sub(r'[^A-Za-z0-9_]+', '', raw)
    return (safe[:max_len] or "transcription").lower()

def truncate_for_sentiment(text: str, max_tokens: int = 512) -> str:
    tokens = sentiment_tokenizer.encode(text, truncation=True, max_length=max_tokens)
    return sentiment_tokenizer.decode(tokens, skip_special_tokens=True)

def chunk_text(text: str, max_chars: int) -> list[str]:
    """
    Split text into chunks of max_chars length without cutting mid-word.
    """
    words = text.split()
    chunks, current = [], ""
    for w in words:
        if len(current) + len(w) + 1 > max_chars:
            chunks.append(current.strip())
            current = w
        else:
            current += " " + w
    if current:
        chunks.append(current.strip())
    return chunks

def main(in_path):
    audio_path = in_path

    model = whisper.load_model("small", device=DEVICE)
    print(f"Transcribing {audio_path} on {DEVICE}...")
    result = model.transcribe(audio_path)
    transcript_text = result["text"]

    # === SUMMARISATION (chunked) ===
    transcript_chunks = chunk_text(transcript_text, max_chars=4000)
    chunk_summaries = []
    for i, chunk in enumerate(transcript_chunks, 1):
        s = summariser(chunk, max_length=40, min_length=10, do_sample=False)[0]["summary_text"]
        chunk_summaries.append(s)
        print(f"Chunk {i} summary: {s}")

    # Meta-summary
    summary_input = " ".join(chunk_summaries)
    final_summary = summariser(summary_input, max_length=50, min_length=15, do_sample=False)[0]["summary_text"]
    print(f"Final Summary: {final_summary}")

    # === SENTIMENT (chunked + truncation) ===
    sentiment_chunks = chunk_text(transcript_text, max_chars=2000)
    chunk_sentiments = []
    sentiment_scores = []

    for i, chunk in enumerate(sentiment_chunks, 1):
        truncated = truncate_for_sentiment(chunk)
        result = sentiment_analyser(truncated)[0]
        score = result["score"] if result["label"] == "POSITIVE" else -result["score"]
        sentiment_scores.append(score)
        chunk_sentiments.append(f"Chunk {i}: {result['label']} ({result['score']:.2f})")

    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)
    sentiment_label = "POSITIVE" if avg_sentiment > 0 else "NEGATIVE"
    sentiment = f"{sentiment_label} ({abs(avg_sentiment):.2f})"
    print(f"Overall Sentiment: {sentiment}")

    # === SAVE OUTPUT ===
    base_dir = os.path.dirname(audio_path) or "."
    fname = safe_filename(final_summary)
    output_path = os.path.join(base_dir, f"{fname}.txt")

    with open(output_path, "w", encoding="utf-8") as f:
        f.write("=== SUMMARY ===\n")
        f.write(final_summary + "\n\n")
        f.write("=== CHUNK SUMMARIES ===\n")
        f.write("\n".join(chunk_summaries) + "\n\n")
        f.write("=== SENTIMENT (Overall) ===\n")
        f.write(sentiment + "\n\n")
        f.write("=== SENTIMENT (Per Chunk) ===\n")
        f.write("\n".join(chunk_sentiments) + "\n\n")
        f.write("=== TRANSCRIPT ===\n")
        f.write(transcript_text)

    print(f"Transcription, summary, and sentiment saved to {output_path}")

In [ ]:
if __name__ == "__main__":
    for i in range(7,79):
        main(f"Recordings/Recording ({i}).m4a")

In [ ]:
if __name__ == "__main__":
    main(f"Recordings/Recording.wav")

In [ ]:
conda install -c conda-forge ffmpeg